# 2 Data cleaning

## 2.1 Import libraries, load dataset

In [39]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

In [40]:
data_path = '../data/accidents_catalonia_2010_2023.csv'
df = pd.read_csv(data_path)
print(f"Dataset loaded successfully from {data_path}")

Dataset loaded successfully from ../data/accidents_catalonia_2010_2023.csv


In [41]:
# save original
df_original = df.copy()

## 2.2 Drop not needed columns

First, we extract month from date

In [42]:
# copy dataset
df_original = df.copy()

# get only month from date
df['Mes'] = pd.to_datetime(df['dat']).dt.month

C:\Users\User\AppData\Local\Temp\ipykernel_3764\1995985882.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Mes'] = pd.to_datetime(df['dat']).dt.month


Now, we delete the following columns:
- dat (we only need year and month)
- pk (we don't need the exact kilometer)
- nomMun (with *nomCom* and *nomDem* is sufficient)
- grupDiaLab (we already have *tipDia*)
- via (too many unique values, and around 10k *Sense especificar*)

- Post-accident variables: D_INFLUIT_BOIRA, D_INFLUIT_CARACT_ENTORN, D_INFLUIT_CIRCULACIO, D_INFLUIT_ESTAT_CLIMA, D_INFLUIT_INTEN_VENT, D_INFLUIT_LLUMINOSITAT, D_INFLUIT_MESU_ESP, D_INFLUIT_OBJ_CALCADA, D_INFLUIT_SOLCS_RASES, D_INFLUIT_VISIBILITAT.

In [43]:
cols_to_remove = ['dat', 'pk', 'nomMun', 'F_ALTRES_UNIT_IMPLICADES', 'F_UNIT_DESC_IMPLICADES', 'D_ACC_AMB_FUGA', 'D_GRAVETAT', 'D_INFLUIT_BOIRA', 'D_INFLUIT_CARACT_ENTORN', 'D_INFLUIT_CIRCULACIO', 'D_INFLUIT_ESTAT_CLIMA', 'D_INFLUIT_INTEN_VENT', 'D_INFLUIT_LLUMINOSITAT', 'D_INFLUIT_MESU_ESP', 'D_INFLUIT_OBJ_CALCADA', 'D_INFLUIT_SOLCS_RASES', 'D_INFLUIT_VISIBILITAT', 'grupDiaLab', 'via']
df = df.drop(columns=cols_to_remove)
print(f"Number of columns after removal: {len(df.columns)}")

Number of columns after removal: 40


## 2.3 Handle missing values

In [44]:
missing_values = df.isnull().sum()
print("Missing Values per Column:")
print(missing_values[missing_values > 0] if missing_values.sum() > 0 else "No missing values found!")
print(f"\nTotal missing values: {missing_values.sum()}")

Missing Values per Column:
C_VELOCITAT_VIA              3062
D_CARACT_ENTORN                41
D_CARRIL_ESPECIAL            1741
D_CIRCULACIO_MESURES_ESP       49
D_REGULACIO_PRIORITAT       17488
D_SENTITS_VIA                4301
D_SUBTIPUS_TRAM             16494
D_TITULARITAT_VIA           12509
D_TRACAT_ALTIMETRIC          8898
dtype: int64

Total missing values: 64583


### VELOCITAT_VIA

Replace incorrect values (999km/h) with NaN

In [45]:
# Replace 999 with NaN
df['C_VELOCITAT_VIA'] = df['C_VELOCITAT_VIA'].replace(999, np.nan)

Percentage of rows with at least 1 death in the dataset

In [46]:
# percentage of rows with deaths in dataset
# rows with at least 1 death
rows_with_deaths = df[df['F_MORTS'] > 0]
print(f"Total records: {len(df)}")
print(f"Records with at least 1 death: {len(rows_with_deaths)}")
print(f"Percentage of records with at least 1 death: {len(rows_with_deaths) / len(df) * 100:.1f}%")

Total records: 24478
Records with at least 1 death: 3093
Percentage of records with at least 1 death: 12.6%


Percentage of rows with at least 1 death in records with invalid speed

In [47]:
# how many records with NAN contain at least 1 death?
speed_missing = df[df['C_VELOCITAT_VIA'].isna()]
print(f"Records with invalid speed: {len(speed_missing)}")
print(f"Deaths in these records: {speed_missing['F_MORTS'].sum()}")
print(f"Percentage of total deaths: {speed_missing['F_MORTS'].sum() / df['F_MORTS'].sum() * 100:.1f}%")
print(f"Percentage of total records: {len(speed_missing) / len(df) * 100:.1f}%")

Records with invalid speed: 4777
Deaths in these records: 192
Percentage of total deaths: 5.7%
Percentage of total records: 19.5%


Most of the records with invalid VELOCITAT_VIA values have no deaths. We already have many records with no deaths. Therefore, we decide to delete them. 

In [48]:
# delete records with invalid speed
print(f"Dataset size before removing records with invalid speed: {len(df)}")
df = df[df['C_VELOCITAT_VIA'].notna()]
print(f"Dataset size after removing records with invalid speed: {len(df)}")

Dataset size before removing records with invalid speed: 24478
Dataset size after removing records with invalid speed: 19701


### Categorical variables - transform to "Sense Especificar"

In [49]:
# Change "Sense especificar" to "Sense Especificar" in D_SENTITS_VIA, D_SUBTIPUS_TRAM, D_TRACAT_ALTIMETRIC
cols_se = ['D_SENTITS_VIA', 'D_SUBTIPUS_TRAM', 'D_TRACAT_ALTIMETRIC']

for col in cols_se:
    df[col] = df[col].replace('Sense especificar', 'Sense Especificar')

In [50]:
cols_to_fill = ['D_CARACT_ENTORN', 'D_CARRIL_ESPECIAL', 'D_CIRCULACIO_MESURES_ESP',
    'D_REGULACIO_PRIORITAT',
    'D_SENTITS_VIA',
    'D_SUBTIPUS_TRAM',
    'D_TITULARITAT_VIA',
    'D_TRACAT_ALTIMETRIC'
]

for col in cols_to_fill:
    df[col] = df[col].fillna('Sense Especificar')

In [51]:
missing_values = df.isnull().sum()
print("Missing Values per Column:")
print(missing_values[missing_values > 0] if missing_values.sum() > 0 else "No missing values found!")
print(f"\nTotal missing values: {missing_values.sum()}")

Missing Values per Column:
No missing values found!

Total missing values: 0


## 2.4 Convert hour to float

In [52]:
df['hor'] = df['hor'].astype(str).str.replace(',', '.').astype(float)

## 2.5 Encode categorical variables

In [53]:
# print unique values of each categorical column
for col in df.select_dtypes(include=['object']).columns:
    print(f"\nUnique values in column '{col}':")
    print(df[col].value_counts())
    print("-"*40)


Unique values in column 'zona':
zona
Zona urbana    10216
Carretera       9485
Name: count, dtype: int64
----------------------------------------

Unique values in column 'nomCom':
nomCom
Barcelonès           3584
Vallès Occidental    2292
Baix Llobregat       1432
Vallès Oriental      1146
Tarragonès            750
Maresme               722
Bages                 688
Selva                 686
Alt Empordà           682
Baix Camp             671
Gironès               652
Segrià                649
Osona                 520
Garraf                475
Alt Penedès           418
Baix Ebre             367
Anoia                 345
Baix Empordà          343
Baix Penedès          336
Montsià               300
Noguera               251
Alt Urgell            238
Garrotxa              217
Alt Camp              189
Berguedà              165
Cerdanya              157
Pla d'Urgell          155
Ribera d'Ebre         129
Pla de l'Estany       121
Segarra               115
Urgell                114
Palla

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19701 entries, 0 to 24476
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Any                        19701 non-null  int64  
 1   zona                       19701 non-null  object 
 2   nomCom                     19701 non-null  object 
 3   nomDem                     19701 non-null  object 
 4   F_MORTS                    19701 non-null  int64  
 5   F_FERITS_GREUS             19701 non-null  int64  
 6   F_FERITS_LLEUS             19701 non-null  int64  
 7   F_VICTIMES                 19701 non-null  int64  
 8   F_UNITATS_IMPLICADES       19701 non-null  int64  
 9   F_VIANANTS_IMPLICADES      19701 non-null  int64  
 10  F_BICICLETES_IMPLICADES    19701 non-null  int64  
 11  F_CICLOMOTORS_IMPLICADES   19701 non-null  int64  
 12  F_MOTOCICLETES_IMPLICADES  19701 non-null  int64  
 13  F_VEH_LLEUGERS_IMPLICADES  19701 non-null  int64  


In [55]:
cat_cols = df.select_dtypes(include=['object']).columns
print("Categorical Columns:")
print(cat_cols)

Categorical Columns:
Index(['zona', 'nomCom', 'nomDem', 'D_BOIRA', 'D_CARACT_ENTORN',
       'D_CARRIL_ESPECIAL', 'D_CIRCULACIO_MESURES_ESP', 'D_CLIMATOLOGIA',
       'D_FUNC_ESP_VIA', 'D_INTER_SECCIO', 'D_LIMIT_VELOCITAT',
       'D_LLUMINOSITAT', 'D_REGULACIO_PRIORITAT', 'D_SENTITS_VIA',
       'D_SUBTIPUS_ACCIDENT', 'D_SUBTIPUS_TRAM', 'D_SUBZONA', 'D_SUPERFICIE',
       'D_TIPUS_VIA', 'D_TITULARITAT_VIA', 'D_TRACAT_ALTIMETRIC', 'D_VENT',
       'grupHor', 'tipAcc', 'tipDia'],
      dtype='object')


In [56]:
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le  # Save encoders for inverse transform later


## 2.6 Feature engineering

Create "mortality" column: 1 if >=1 death, 0 else.

In [57]:
# Create "mortality" column: 1 if >=1 death, 0 else.
df['Mortalitat'] = (df['F_MORTS'] > 0).astype(int)

In [58]:
df.head()

,Any,zona,nomCom,nomDem,F_MORTS,F_FERITS_GREUS,F_FERITS_LLEUS,F_VICTIMES,F_UNITATS_IMPLICADES,F_VIANANTS_IMPLICADES,...,D_TIPUS_VIA,D_TITULARITAT_VIA,D_TRACAT_ALTIMETRIC,D_VENT,hor,grupHor,tipAcc,tipDia,Mes,Mortalitat
0,2010,1,42,0,0,1,0,1,2,0,...,5,5,4,0,23.33,1,4,1,1,0
1,2010,0,34,2,0,1,3,4,1,0,...,4,2,2,0,1.00,1,5,0,10,0
2,2010,0,20,1,1,0,2,3,4,0,...,4,2,3,0,15.27,2,4,1,5,1
3,2010,1,13,0,0,2,7,9,2,0,...,5,5,4,0,22.30,1,4,2,8,0
4,2010,1,13,0,0,1,0,1,1,0,...,5,5,4,0,17.45,2,2,3,5,0


## 2.7 Save label encoders and cleaned dataset

In [59]:
# TODO